In [ ]:
import pandas as pd
import numpy as np
from sklearn import tree, preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
so2AllData = pd.DataFrame()

**Data Preprocessing**

In [ ]:
def processData(filename):
  # Read input file
  df2007 = pd.read_csv(filename, encoding="utf8")
  #Exctract date,month and year from "Date Local"
  df2007['month'] = pd.DatetimeIndex(df2007['Date Local']).month
  df2007['day'] = pd.DatetimeIndex(df2007['Date Local']).day
  df2007['year'] = pd.DatetimeIndex(df2007['Date Local']).year
  #Remove duplicates and null values
  df2007 = df2007.dropna()
  df2007 = df2007.drop_duplicates()
  #Keep State code in int format only
  df2007['State Code'] = pd.to_numeric(df2007['State Code'])
  #Remove irrelevant columns
  df2007 = df2007.drop(columns=['Observation Percent'])
  #Encode event type
  le = preprocessing.LabelEncoder()
  df2007['Event Type'] = le.fit_transform(df2007['Event Type'])
  return df2007

**Read files from 2000-2017 and form trainind data**

In [ ]:
so2AllData = pd.concat(
    map(processData, ['/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2000.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2001.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2002.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2003.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2004.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2005.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2006.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2007.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2008.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2009.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2010.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2011.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2012.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2013.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2014.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2015.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2016.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2017.csv'
                      ]), ignore_index=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.


In [ ]:
so2AllData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2439765 entries, 0 to 2439764
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   State Code           int64  
 1   County Code          int64  
 2   Site Num             int64  
 3   Parameter Code       int64  
 4   POC                  int64  
 5   Latitude             float64
 6   Longitude            float64
 7   Datum                object 
 8   Parameter Name       object 
 9   Sample Duration      object 
 10  Pollutant Standard   object 
 11  Date Local           object 
 12  Units of Measure     object 
 13  Event Type           int64  
 14  Observation Count    int64  
 15  Arithmetic Mean      float64
 16  1st Max Value        float64
 17  1st Max Hour         int64  
 18  AQI                  float64
 19  Method Code          float64
 20  Method Name          object 
 21  Local Site Name      object 
 22  Address              object 
 23  State Name           object 
 24

In [ ]:
so2AllData.to_csv('/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/S02_18yearDailySummary.csv')


In [ ]:
so2AllData.groupby(['State Code','County Code','Site Num','Date Local']).agg({'AQI':'count'})

AQI
State Code County Code Site Num Date Local     
1          33          1002     2002-04-01    1
                                2002-04-02    1
                                2002-04-03    1
                                2002-04-04    1
                                2002-04-05    1
...                                         ...
72         127         9        2005-11-12    1
                                2005-11-13    1
                                2005-11-14    1
                                2005-11-15    1
                                2005-11-16    1

[2421656 rows x 1 columns]

In [ ]:
so2AllData = so2AllData.select_dtypes(exclude=['object']) 
 

**Taking daily summary of 2000-2018 as training data. and 2019 & 2020 data as test data**

In [ ]:
X_train = so2AllData.drop(['AQI'], axis=1).values
Y_train = so2AllData['AQI'].values

**Predicted AQI for SO2 pollutant using Gradient Boosting Regressor with 98.7% train accuracy**

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
grb = GradientBoostingRegressor(n_estimators=40)
grb.fit(X_train, Y_train)
print(grb.score(X_train, Y_train))

0.9877014291740575


**Read data from 2018-2020 and form test data**

In [ ]:
testdata = pd.DataFrame()

In [ ]:
testdata = pd.concat(
    map(processData, [
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2018.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2019.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2020.csv'
                      ]), ignore_index=True)

In [ ]:

testdata = testdata.select_dtypes(exclude=['object'])

In [ ]:
X_test = testdata.drop(['AQI'], axis=1).values
Y_test = testdata['AQI'].values

**Predicted AQI with 98.2% test accuracy for SO2 pollutant**

In [ ]:
accuracy = grb.score(X_test, Y_test)
print("Accuracy on test data: %.6f" % accuracy)

Accuracy on test data: 0.982016


In [ ]:
validationData = pd.DataFrame()

**Split training data into training set and validation set**

In [ ]:
validationData = validationData.append(dailySummaryof21years("/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/data/daily_42401_2017.csv"))

In [ ]:

validationData = validationData.select_dtypes(exclude=['object'])

In [ ]:
X_validation = validationData.drop(['AQI'], axis=1).values
Y_validation = validationData['AQI'].values

**Tried to tune parameter using GridsearchCV and validation dataset. However, these parameters are bringing test accuracy down.**

In [ ]:
from sklearn.model_selection import GridSearchCV
grid={"n_estimators":[10,20,30,40,100],"max_depth":[3,5,10,50]}
gbrGrid=GradientBoostingRegressor()
gbrGridCV=GridSearchCV(gbrGrid,grid,cv=3)
gbrGridCV.fit(X_validation, Y_validation)

print("tuned hpyerparameters :(best parameters) ",gbrGridCV.best_params_)
print("accuracy :",gbrGridCV.best_score_)

tuned hpyerparameters :(best parameters)  {'max_depth': 50, 'n_estimators': 100}
accuracy : 0.9765630770671984


In [ ]:
predicted_AQI = grb.predict(X_test)

In [ ]:
predictedResult = testdata.drop(['AQI'], axis=1)
predictedResult = predictedResult.reset_index()
predictedResult = predictedResult.join(pd.DataFrame({'AQI predicted':predicted_AQI}))

In [ ]:
predictedResult = predictedResult.drop(columns=['index'])

**Write predited AQI into a file for further analysis**

In [ ]:
newSO = predictedResult[['State Code','County Code','Site Num','Latitude','Longitude','month','day','year','AQI predicted']]

In [ ]:
newSO.to_csv("/content/drive/MyDrive/FA21 CMPE 255 Term Project/SO2/SO2_AQI_Predicted")

In [ ]:
predictedResult.sort_values(by='AQI predicted', ascending=False)

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Event Type,Observation Count,Arithmetic Mean,1st Max Value,1st Max Hour,Method Code,month,day,year,AQI predicted
30021,15,1,7,42401,1,19.420506,-155.287853,1,23,104.478261,388.0,10,60.0,6,14,2018,195.007423
30856,15,1,2016,42401,1,19.203900,-155.480183,2,22,51.772727,759.4,16,60.0,3,27,2018,195.007423
31032,15,1,2016,42401,1,19.203900,-155.480183,1,23,66.795652,309.2,8,60.0,7,19,2018,195.007423
31366,15,1,2020,42401,1,19.117561,-155.778136,1,24,155.720833,771.0,20,60.0,5,25,2018,195.007423
29659,15,1,5,42401,1,19.430800,-155.257800,1,23,79.826087,548.0,10,60.0,6,3,2018,195.007423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228401,37,13,151,42401,1,35.428000,-76.739900,1,23,0.000000,0.0,0,60.0,10,30,2019,0.379239
228402,37,13,151,42401,1,35.428000,-76.739900,1,23,0.021739,0.3,12,60.0,10,31,2019,0.379239
228404,37,13,151,42401,1,35.428000,-76.739900,1,23,0.073913,0.2,11,60.0,11,2,2019,0.379239
228405,37,13,151,42401,1,35.428000,-76.739900,1,23,0.165217,0.6,9,60.0,11,3,2019,0.379239


AQI for each county per month

In [ ]:
countyResult = predictedResult.groupby(['State Code','County Code','year','month']).agg({"AQI predicted":['mean']})
countyResult.columns = ['AQI']
countyResult.reset_index()

,State Code,County Code,year,month,AQI
0,1,73,2018,1,8.494709
1,1,73,2018,2,6.796796
2,1,73,2018,3,14.599073
3,1,73,2018,4,10.181082
4,1,73,2018,5,12.384306
...,...,...,...,...,...
9617,72,33,2020,6,1.844243
9618,72,33,2020,7,3.077011
9619,72,33,2020,8,4.222428
9620,72,33,2020,9,4.777004


In [ ]:
column = predictedResult['AQI predicted']
max_value = column.max()
max_value

195.00742323765056